# Financial Agents System - Integrated Notebook

This notebook combines three financial agent components into a single integrated system:

1. **Knowledge Agent**: Retrieves information about financial products
2. **Card Agent**: Executes card-related actions (activate, deactivate)
3. **Orchestrator Agent**: Coordinates between the other agents based on user intent

Each section can be run independently for testing or debugging. The notebook is structured to allow for both component-level and system-level testing.

## 1. Common Setup and Imports

First, we'll import all the required libraries for all three agents.

In [10]:
import os
import json
import requests
from typing import TypedDict, Annotated, Sequence, Literal
import operator
from langgraph.graph import StateGraph, END
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API key from environment variable
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in environment variables. Please check your .env file.")

# Initialize OpenAI client with API key
client = OpenAI(api_key=OPENAI_API_KEY)

# Base URL for the Card Service API (adjust as needed)
CARD_API_BASE_URL = "http://localhost:8080/api/cards"

print("Common setup complete!")

Common setup complete!


# 2. Knowledge Agent

The Knowledge Agent retrieves information from a vector database (simulated in this notebook) and uses LLMs to generate helpful responses about financial products.

## 2.1 Knowledge Agent State Definition

In [ ]:
class KnowledgeAgentState(TypedDict):
    query: str
    search_results: list[dict]  # Results from vector DB
    response: str
    error: str | None

print("KnowledgeAgentState defined!")

## 2.2 Knowledge Retrieval Function
This function simulates searching a vector database for relevant information.

In [ ]:
def retrieve_knowledge(state: KnowledgeAgentState) -> KnowledgeAgentState:
    """Retrieves relevant knowledge from the vector database based on the query."""
    print(f"--- Knowledge Agent: Retrieving knowledge for query: {state['query']} ---")
    query = state['query']
    search_results = []
    error = None
    try:
        # Placeholder: Implement actual vector search logic
        # response = vector_db_client.search(...)
        print("Placeholder: Performing vector search in OpenSearch/Vector DB...")
        
        # Simulate finding relevant product info from our synthetic data
        if "hsa" in query.lower() or "health savings" in query.lower():
            search_results.append({"id": "HSA001", "text": "Details about Health Savings Account...", "score": 0.9})
        elif "fsa" in query.lower() or "flexible spending" in query.lower():
            search_results.append({"id": "FSA001", "text": "Details about Flexible Spending Account...", "score": 0.88})
        elif "prepaid" in query.lower():
            search_results.append({"id": "PREPAID001", "text": "Details about General Purpose Prepaid Card...", "score": 0.85})
        else:
            search_results.append({"id": "GENERAL", "text": "General financial product information...", "score": 0.7})
        
        print(f"Found {len(search_results)} potential results.")

    except Exception as e:
        print(f"Error during knowledge retrieval: {e}")
        error = f"Failed to retrieve knowledge: {e}"
        search_results = []

    return {**state, "search_results": search_results, "error": error}

print("Knowledge retrieval function defined!")

## 2.3 Knowledge Response Generation
This function generates a response based on the retrieved knowledge using OpenAI.

In [11]:
def generate_response(state: KnowledgeAgentState) -> KnowledgeAgentState:
    """Generates a response based on the retrieved knowledge using OpenAI."""
    print("--- Knowledge Agent: Generating response ---")
    if state['error']:
        return {**state, "response": f"Sorry, I encountered an error: {state['error']}"}
    if not state['search_results']:
         return {**state, "response": "I couldn't find specific information for your query."}

    query = state['query']
    context = "\n".join([f"Result {i+1}: {res.get('text', '')} (ID: {res.get('id', 'N/A')})" for i, res in enumerate(state['search_results'])])

    prompt = f"You are a helpful financial knowledge assistant. Answer the user's query based *only* on the provided context.\n\nContext:\n{context}\n\nUser Query: {query}\n\nAnswer:"

    try:
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo", # Or specify another model like gpt-4
            messages=[
                {"role": "system", "content": "You are a helpful financial knowledge assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        response = completion.choices[0].message.content
        print(f"Generated response: {response[:100]}...") # Log snippet
    except Exception as e:
        print(f"Error during OpenAI completion: {e}")
        response = f"Sorry, I encountered an error while generating the response: {e}"
        state['error'] = response # Store the error

    return {**state, "response": response}

print("Response generation function defined!")

Response generation function defined!


## 2.4 Knowledge Agent Graph Construction
This section defines the LangGraph workflow for the Knowledge Agent.

In [12]:
def build_knowledge_agent():
    """Builds and returns the knowledge agent workflow."""
    # Create graph
    knowledge_workflow = StateGraph(KnowledgeAgentState)
    
    # Add nodes
    knowledge_workflow.add_node("retrieve", retrieve_knowledge)
    knowledge_workflow.add_node("generate", generate_response)
    
    # Define edges
    knowledge_workflow.set_entry_point("retrieve")
    knowledge_workflow.add_edge("retrieve", "generate")
    knowledge_workflow.add_edge("generate", END)
    
    # Compile graph
    return knowledge_workflow.compile()

# Build the agent
knowledge_agent_app = build_knowledge_agent()
print("Knowledge agent workflow built!")

Knowledge agent workflow built!


## 2.5 Knowledge Agent Testing Function
This helper function lets us test the Knowledge Agent in isolation.

In [13]:
def test_knowledge_agent(query):
    """Test the knowledge agent with the given query."""
    inputs = {
        "query": query,
        "search_results": [],
        "response": "",
        "error": None
    }
    
    print(f"Testing Knowledge Agent with query: {query}")
    result = knowledge_agent_app.invoke(inputs)
    
    print(f"\n--- Final Knowledge Agent Result ---")
    print(f"Query: {result['query']}")
    print(f"\nResponse: {result['response']}")
    if result['error']:
        print(f"\nError: {result['error']}")
    
    return result

### Test Knowledge Agent
Uncomment to run a test query against the Knowledge Agent:

In [16]:
# Test with an HSA query
hsa_result = test_knowledge_agent("Tell me about hsa")

Testing Knowledge Agent with query: Tell me about hsa
--- Knowledge Agent: Retrieving knowledge for query: Tell me about hsa ---
Placeholder: Performing vector search in OpenSearch/Vector DB...
Found 1 potential results.
--- Knowledge Agent: Generating response ---
Generated response: Details about Health Savings Account can be found in Result 1....

--- Final Knowledge Agent Result ---
Query: Tell me about hsa

Response: Details about Health Savings Account can be found in Result 1.


## 2.6 Knowledge Agent Debug Utilities
Additional utility functions for debugging the Knowledge Agent.

In [ ]:
def simulate_vector_search(query, detailed=False):
    """Simulates a vector search with more detailed mock data for testing."""
    print(f"Simulating vector search for: {query}")
    
    # More detailed mock data for testing
    detailed_results = {
        "hsa": {
            "id": "HSA001",
            "text": "Health Savings Accounts (HSAs) are tax-advantaged accounts available to individuals enrolled in high-deductible health plans. Contributions are tax-deductible, grow tax-free, and withdrawals for qualified medical expenses are also tax-free. For 2023, contribution limits are $3,850 for individuals and $7,750 for families.",
            "score": 0.92
        },
        "fsa": {
            "id": "FSA001", 
            "text": "Flexible Spending Accounts (FSAs) allow employees to set aside pre-tax dollars for qualified healthcare expenses. Unlike HSAs, FSAs typically have a 'use it or lose it' policy where funds must be used within the plan year or grace period. The 2023 contribution limit is $3,050.",
            "score": 0.89
        },
        "prepaid": {
            "id": "PREPAID001", 
            "text": "General Purpose Prepaid Cards allow users to load funds in advance and use for transactions anywhere the card network is accepted. They typically have fees for activation ($5-10), monthly maintenance ($5-7), ATM withdrawals ($2-3), and reloading ($3-5). They're useful for budgeting and don't require credit checks.",
            "score": 0.88
        }
    }
    
    # Basic search logic
    results = []
    if "hsa" in query.lower() or "health savings" in query.lower():
        results.append(detailed_results["hsa"] if detailed else {"id": "HSA001", "text": "Details about Health Savings Account...", "score": 0.9})
    elif "fsa" in query.lower() or "flexible spending" in query.lower():
        results.append(detailed_results["fsa"] if detailed else {"id": "FSA001", "text": "Details about Flexible Spending Account...", "score": 0.88})
    elif "prepaid" in query.lower():
        results.append(detailed_results["prepaid"] if detailed else {"id": "PREPAID001", "text": "Details about General Purpose Prepaid Card...", "score": 0.85})
    else:
        results.append({"id": "GENERAL", "text": "General financial product information...", "score": 0.7})
    
    print(f"Found {len(results)} results.")
    for result in results:
        print(f"- {result['id']}: {result['text'][:50]}... (score: {result['score']})")
    
    return results

In [ ]:
# Uncomment to test the simulated vector search
# detailed_search_results = simulate_vector_search("Tell me about HSA accounts", detailed=True)

# 3. Card Agent

The Card Agent handles card-related actions such as activation and deactivation, communicating with a card service API.

## 3.1 Card Agent State Definition

In [17]:
class CardAgentState(TypedDict):
    action: str            # e.g., "activate", "deactivate"
    card_number: str       # Or other identifier
    parameters: dict       # Any additional parameters needed for the API call
    api_response: dict | None  # Response from the card API
    confirmation_message: str  # User-facing message
    error: str | None

print("CardAgentState defined!")

CardAgentState defined!


## 3.2 Card API Call Function
This function handles the HTTP requests to the card service.

In [18]:
def call_card_api(action: str, card_number: str, parameters: dict) -> dict:
    """Calls the Card Service API."""
    url = f"{CARD_API_BASE_URL}/{action}"
    headers = {'Content-Type': 'application/json'}
    payload = {"cardNumber": card_number, **parameters}

    print(f"--- Card Agent: Calling API: {action} for card ending {card_number[-4:]} ---")
    print(f"URL: {url}")
    print(f"Payload: {payload}")

    # For testing/debugging - uncomment to skip actual API call
    # return {"success": True, "message": f"SIMULATED {action} success"}
    
    try:
        if action == "activate":
            response = requests.post(url, json=payload, headers=headers, timeout=10)
        elif action == "deactivate":
            response = requests.post(url, json=payload, headers=headers, timeout=10)
        else:
            return {"success": False, "message": f"Unsupported action: {action}"}

        response.raise_for_status()
        api_response_data = response.json()
        print(f"API Success: {api_response_data}")
        return {"success": True, **api_response_data}

    except requests.exceptions.RequestException as e:
        error_message = f"API call failed: {e}"
        print(f"API Error: {error_message}")
        try:
            error_details = response.json()
            error_message += f" Details: {error_details}"
        except:
            pass
        return {"success": False, "message": error_message}
    except Exception as e:
        error_message = f"An unexpected error occurred during API call: {e}"
        print(f"API Error: {error_message}")
        return {"success": False, "message": error_message}

print("Card API call function defined!")

Card API call function defined!


## 3.3 Card Action Execution
This function processes the API results and updates the state.

In [19]:
def execute_card_action(state: CardAgentState) -> CardAgentState:
    """Executes the requested card action by calling the API."""
    action = state['action']
    card_number = state['card_number']
    parameters = state['parameters']

    print(f"Executing card action: {action}")
    api_result = call_card_api(action, card_number, parameters)

    confirmation_message = ""
    error = None

    if api_result.get("success"):
        confirmation_message = api_result.get("message", f"Card {action} request processed successfully.")
        print(f"Card Action Success: {confirmation_message}")
    else:
        error = api_result.get("message", "An unknown error occurred during the card action.")
        confirmation_message = f"Failed to {action} card. Error: {error}"
        print(f"Card Action Failed: {error}")

    return {**state, "api_response": api_result, "confirmation_message": confirmation_message, "error": error}

print("Card action execution function defined!")

Card action execution function defined!


## 3.4 Card Agent Graph Construction
This section defines the LangGraph workflow for the Card Agent.

In [20]:
def build_card_agent():
    """Builds and returns the card agent workflow."""
    # Create graph
    card_workflow = StateGraph(CardAgentState)
    
    # Add node
    card_workflow.add_node("execute", execute_card_action)
    
    # Define edges
    card_workflow.set_entry_point("execute")
    card_workflow.add_edge("execute", END)
    
    # Compile graph
    return card_workflow.compile()

# Build the agent
card_agent_app = build_card_agent()
print("Card agent workflow built!")

Card agent workflow built!


## 3.5 Card Agent Testing Function
This helper function lets us test the Card Agent in isolation.

In [21]:
def test_card_agent(action, card_number, **parameters):
    """Test the card agent with the given inputs."""
    inputs = {
        "action": action,
        "card_number": card_number,
        "parameters": parameters,
        "api_response": None,
        "confirmation_message": "",
        "error": None
    }
    
    print(f"Testing Card Agent ({action})...")
    result = card_agent_app.invoke(inputs)
    
    print(f"\n--- Final Card Agent Result ({action}) ---")
    for key, value in result.items():
        print(f"{key}: {value}")
    
    return result

### Test Card Agent
Uncomment to run a test action with the Card Agent:

In [22]:
# Activate a card
activate_result = test_card_agent(
    "activate", 
    "1111222233334444", 
    cvv="123", 
    expiryDate="04/26"
)

Testing Card Agent (activate)...
Executing card action: activate
--- Card Agent: Calling API: activate for card ending 4444 ---
URL: http://localhost:8080/api/cards/activate
Payload: {'cardNumber': '1111222233334444', 'cvv': '123', 'expiryDate': '04/26'}
API Success: {'success': True, 'message': 'Card activated successfully', 'cardNumber': '****4444'}
Card Action Success: Card activated successfully

--- Final Card Agent Result (activate) ---
action: activate
card_number: 1111222233334444
parameters: {'cvv': '123', 'expiryDate': '04/26'}
api_response: {'success': True, 'message': 'Card activated successfully', 'cardNumber': '****4444'}
confirmation_message: Card activated successfully
error: None


In [ ]:
# Deactivate a card
# deactivate_result = test_card_agent(
#     "deactivate", 
#     "1111222233334444", 
#     reason="Lost card"
# )

## 3.6 Card Agent Debug Utilities
Additional utility functions for debugging the Card Agent.

In [ ]:
def inspect_api_request(action: str, card_number: str, parameters: dict):
    """Displays the API request that would be made without actually making it."""
    url = f"{CARD_API_BASE_URL}/{action}"
    headers = {'Content-Type': 'application/json'}
    payload = {"cardNumber": card_number, **parameters}
    
    print("=== API Request Details (No request sent) ===")
    print(f"URL: {url}")
    print(f"Method: POST")
    print(f"Headers: {headers}")
    print(f"Payload: {payload}")
    
    return {"url": url, "method": "POST", "headers": headers, "payload": payload}

In [ ]:
# Uncomment to inspect a request without sending it
# inspect_api_request(
#     "activate", 
#     "1111222233334444", 
#     {"cvv": "123", "expiryDate": "12/25"}
# )

# 4. Orchestrator Agent

The Orchestrator Agent is the main entry point for users. It determines the user's intent and routes the request to the appropriate agent.

## 4.1 Orchestrator State Definition

In [23]:
class OrchestratorState(TypedDict):
    user_query: str
    intent: Literal["knowledge", "card_action", "unknown", "error"]
    card_action_details: CardAgentState | None  # Details needed for card agent
    knowledge_agent_response: str | None
    card_agent_response: str | None
    final_response: str
    error: str | None

print("OrchestratorState defined!")

OrchestratorState defined!


## 4.2 Intent Classification Function
This function uses OpenAI to determine the user's intent.

In [24]:
def classify_intent(state: OrchestratorState) -> OrchestratorState:
    """Classifies the user's intent using OpenAI."""
    print(f"--- Orchestrator: Classifying intent for query: {state['user_query']} ---")
    query = state['user_query']
    intent = "unknown"
    error = None
    card_action_details = None

    prompt = f"""Classify the user's intent based on their query. Choose one: 'knowledge', 'card_action', or 'unknown'.
    - 'knowledge': User is asking for information (e.g., 'What is an HSA?', 'Tell me about prepaid cards').
    - 'card_action': User wants to perform an action on a card (e.g., 'Activate my card', 'Deactivate card ending in 1234').
    - 'unknown': The intent is unclear or not related to finance/cards.

    If the intent is 'card_action', also extract the action (e.g., 'activate', 'deactivate') and any card identifiers (like last 4 digits or full number if provided). Format the output as JSON: {{"intent": "...". "action": "...", "card_identifier": "..."}}
    If the intent is 'knowledge' or 'unknown', format as JSON: {{"intent": "..."}}

    User Query: "{query}"

    JSON Output:"""

    try:
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",  # Use a model suitable for classification
            messages=[
                {"role": "system", "content": "You are an intent classification expert for financial services."},
                {"role": "user", "content": prompt}
            ],
            response_format={"type": "json_object"}  # Request JSON output if model supports
        )
        result_json = json.loads(completion.choices[0].message.content)
        intent = result_json.get("intent", "unknown")
        print(f"Classified intent: {intent}")

        if intent == "card_action":
            action = result_json.get("action")
            card_identifier = result_json.get("card_identifier")
            if action and card_identifier:
                # In a real system, you'd need more robust extraction and potentially clarification
                card_action_details = {
                    "action": action,
                    "card_number": card_identifier,  # May need validation/lookup
                    "parameters": {},  # Placeholder for additional params if needed
                    # Initialize other CardAgentState fields as None or empty
                    "api_response": None,
                    "confirmation_message": "",
                    "error": None
                }
                print(f"Extracted card action details: {card_action_details}")
            else:
                print("Warning: card_action intent detected but details missing.")
                intent = "unknown"  # Fallback if details can't be extracted
                error = "Could not extract necessary details for the card action."

    except Exception as e:
        print(f"Error during intent classification: {e}")
        intent = "error"
        error = f"Failed to classify intent: {e}"

    return {**state, "intent": intent, "card_action_details": card_action_details, "error": error}

print("Intent classification function defined!")

Intent classification function defined!


## 4.3 Agent Routing Functions
These functions route the user query to the appropriate agent based on intent.

In [25]:
def route_to_knowledge_agent(state: OrchestratorState) -> OrchestratorState:
    """Invokes the Knowledge Agent."""
    print("--- Orchestrator: Routing to Knowledge Agent ---")
    knowledge_input = {"query": state['user_query']}
    knowledge_result = knowledge_agent_app.invoke(knowledge_input)
    
    response = knowledge_result.get("response", "Knowledge agent did not provide a response.")
    error = knowledge_result.get("error")
    print(f"Knowledge Agent Result: {response[:100]}... Error: {error}")
    return {**state, "knowledge_agent_response": response, "error": error}

In [26]:
def route_to_card_agent(state: OrchestratorState) -> OrchestratorState:
    """Invokes the Card Agent."""
    print("--- Orchestrator: Routing to Card Agent ---")
    if not state['card_action_details']:
        error = "Cannot route to card agent: missing action details."
        print(error)
        return {**state, "card_agent_response": "Internal error: Missing card action details.", "error": error}

    card_input = state['card_action_details']
    card_result = card_agent_app.invoke(card_input)
    
    response = card_result.get("confirmation_message", "Card agent did not provide a response.")
    error = card_result.get("error")
    print(f"Card Agent Result: {response[:100]}... Error: {error}")
    return {**state, "card_agent_response": response, "error": error}

print("Agent routing functions defined!")

Agent routing functions defined!


## 4.4 Response Formatting Function
This function formats the final response based on the agent responses.

In [27]:
def format_final_response(state: OrchestratorState) -> OrchestratorState:
    """Formats the final response based on which agent was called."""
    print("--- Orchestrator: Formatting final response ---")
    if state['error'] and not (state['knowledge_agent_response'] or state['card_agent_response']):
        final_response = f"Sorry, I encountered an error processing your request: {state['error']}"
    elif state['intent'] == 'knowledge':
        final_response = state['knowledge_agent_response'] or "I couldn't retrieve the information."
    elif state['intent'] == 'card_action':
        final_response = state['card_agent_response'] or "The card action request could not be completed."
    else:
        final_response = "I'm not sure how to handle that request. Can you please rephrase?"

    # Append error messages if they occurred within agents but a response was still generated
    if state['error'] and (state['knowledge_agent_response'] or state['card_agent_response']):
       final_response += f"\n(Note: An error occurred during processing: {state['error']})"

    print(f"Final Response: {final_response[:100]}...")
    return {**state, "final_response": final_response}

print("Response formatting function defined!")

Response formatting function defined!


## 4.5 Conditional Edge Logic Function
This function determines the next step based on the classified intent.

In [28]:
def decide_route(state: OrchestratorState) -> Literal["knowledge", "card_action", "end_error", "end_unknown"]:
    """Determines the next step based on the classified intent."""
    print(f"--- Orchestrator: Deciding route based on intent: {state['intent']} ---")
    if state['error'] and state['intent'] == 'error':
        return "end_error"
    elif state['intent'] == 'knowledge':
        return "knowledge"
    elif state['intent'] == 'card_action':
        if state['card_action_details']:  # Check if details were extracted
             return "card_action"
        else:
             print("Routing to end_unknown due to missing card details despite intent.")
             return "end_unknown"  # Treat as unknown if details missing
    else:  # unknown
        return "end_unknown"

print("Route decision function defined!")

Route decision function defined!


## 4.6 Orchestrator Graph Construction
This section defines the LangGraph workflow for the Orchestrator Agent.

In [29]:
def build_orchestrator():
    """Builds and returns the orchestrator workflow."""
    # Create graph
    workflow = StateGraph(OrchestratorState)
    
    # Add nodes
    workflow.add_node("classify_intent", classify_intent)
    workflow.add_node("knowledge_agent", route_to_knowledge_agent)
    workflow.add_node("card_agent", route_to_card_agent)
    workflow.add_node("format_response", format_final_response)
    
    # Define edges
    workflow.set_entry_point("classify_intent")
    
    # Conditional routing after classification
    workflow.add_conditional_edges(
        "classify_intent",
        decide_route,
        {
            "knowledge": "knowledge_agent",
            "card_action": "card_agent",
            "end_error": "format_response",  # Go directly to formatting for critical errors
            "end_unknown": "format_response"  # Go directly to formatting for unknown intent
        }
    )
    
    # Edges from agents to final formatting
    workflow.add_edge("knowledge_agent", "format_response")
    workflow.add_edge("card_agent", "format_response")
    
    # End after formatting
    workflow.add_edge("format_response", END)
    
    # Compile graph
    return workflow.compile()

# Build the orchestrator
orchestrator_app = build_orchestrator()
print("Orchestrator workflow built!")

Orchestrator workflow built!


## 4.7 Orchestrator Testing Function
This helper function lets us test the complete Orchestrator Agent.

In [30]:
def test_orchestrator(query):
    """Test the orchestrator with the given query."""
    inputs = {
        "user_query": query,
        "intent": "unknown",
        "card_action_details": None,
        "knowledge_agent_response": None,
        "card_agent_response": None,
        "final_response": "",
        "error": None
    }
    
    print(f"Testing Orchestrator with query: {query}")
    result = orchestrator_app.invoke(inputs)
    
    print(f"\n--- Final Orchestrator Result ---")
    print(f"Query: {result['user_query']}")
    print(f"Intent: {result['intent']}")
    print(f"\nFinal Response: {result['final_response']}")
    if result['error']:
        print(f"\nError: {result['error']}")
    
    return result

### Test Orchestrator
Uncomment to run tests with the Orchestrator:

In [31]:
# Test with a knowledge query
knowledge_result = test_orchestrator("What are the benefits of an HSA account?")

Testing Orchestrator with query: What are the benefits of an HSA account?
--- Orchestrator: Classifying intent for query: What are the benefits of an HSA account? ---
Classified intent: knowledge
--- Orchestrator: Deciding route based on intent: knowledge ---
--- Orchestrator: Routing to Knowledge Agent ---
--- Knowledge Agent: Retrieving knowledge for query: What are the benefits of an HSA account? ---
Placeholder: Performing vector search in OpenSearch/Vector DB...
Found 1 potential results.
--- Knowledge Agent: Generating response ---
Generated response: Health Savings Accounts (HSAs) offer tax advantages such as contributions being made with pre-tax do...
Knowledge Agent Result: Health Savings Accounts (HSAs) offer tax advantages such as contributions being made with pre-tax do... Error: None
--- Orchestrator: Formatting final response ---
Final Response: Health Savings Accounts (HSAs) offer tax advantages such as contributions being made with pre-tax do...

--- Final Orchestrator 

In [32]:
# Test with a card action query
card_result = test_orchestrator("Please activate my card ending in 3456")

Testing Orchestrator with query: Please activate my card ending in 3456
--- Orchestrator: Classifying intent for query: Please activate my card ending in 3456 ---
Classified intent: card_action
Extracted card action details: {'action': 'activate', 'card_number': '3456', 'parameters': {}, 'api_response': None, 'confirmation_message': '', 'error': None}
--- Orchestrator: Deciding route based on intent: card_action ---
--- Orchestrator: Routing to Card Agent ---
Executing card action: activate
--- Card Agent: Calling API: activate for card ending 3456 ---
URL: http://localhost:8080/api/cards/activate
Payload: {'cardNumber': '3456'}
API Error: API call failed: 400 Client Error:  for url: http://localhost:8080/api/cards/activate
Card Action Failed: API call failed: 400 Client Error:  for url: http://localhost:8080/api/cards/activate Details: {'success': False, 'message': 'Missing required fields for activation (cardNumber, cvv, expiryDate).', 'cardNumber': '****'}
Card Agent Result: Failed 

In [33]:
# Test with an unknown query
unknown_result = test_orchestrator("What is the weather forecast for tomorrow?")

Testing Orchestrator with query: What is the weather forecast for tomorrow?
--- Orchestrator: Classifying intent for query: What is the weather forecast for tomorrow? ---
Classified intent: unknown
--- Orchestrator: Deciding route based on intent: unknown ---
--- Orchestrator: Formatting final response ---
Final Response: I'm not sure how to handle that request. Can you please rephrase?...

--- Final Orchestrator Result ---
Query: What is the weather forecast for tomorrow?
Intent: unknown

Final Response: I'm not sure how to handle that request. Can you please rephrase?


## 4.8 Step-By-Step Debugging Function
This function manually walks through the orchestrator steps for detailed debugging.

In [ ]:
def simulate_orchestrator_step_by_step(query):
    """Manually walks through the orchestrator steps for detailed debugging."""
    print(f"=== STEP-BY-STEP ORCHESTRATOR SIMULATION ===")
    print(f"Query: '{query}'")
    
    # Initialize state
    state = {
        "user_query": query,
        "intent": "unknown",
        "card_action_details": None,
        "knowledge_agent_response": None,
        "card_agent_response": None,
        "final_response": "",
        "error": None
    }
    
    # Step 1: Classify intent
    print("\n--- STEP 1: Intent Classification ---")
    state = classify_intent(state)
    print(f"Intent: {state['intent']}")
    if state['card_action_details']:
        print(f"Card action: {state['card_action_details']['action']}")
        print(f"Card number: {state['card_action_details']['card_number']}")
    
    # Step 2: Route based on intent
    print("\n--- STEP 2: Routing ---")
    route = decide_route(state)
    print(f"Route decision: {route}")
    
    # Step 3: Process with appropriate agent
    print("\n--- STEP 3: Agent Processing ---")
    if route == "knowledge":
        state = route_to_knowledge_agent(state)
        print(f"Knowledge response: {state['knowledge_agent_response']}")
    elif route == "card_action":
        state = route_to_card_agent(state)
        print(f"Card response: {state['card_agent_response']}")
    else:
        print(f"Skipping agent processing, going to final response formatting")
    
    # Step 4: Format final response
    print("\n--- STEP 4: Final Response Formatting ---")
    state = format_final_response(state)
    print(f"Final response: {state['final_response']}")
    
    print("\n=== SIMULATION COMPLETE ===")
    return state

In [ ]:
# Uncomment to run a step-by-step simulation
# detailed_result = simulate_orchestrator_step_by_step("What is an HSA account?")

# 5. Integrated System Testing

This section provides examples of testing the entire system with different types of user queries.

## 5.1 Run the Complete System

In [ ]:
def process_user_query(query):
    """Process a user query through the entire system and return a human-friendly response."""
    print(f"Processing user query: '{query}'")
    
    # Use the orchestrator to process the query
    result = test_orchestrator(query)
    
    # Return the final response
    return result['final_response']

## 5.2 Example Queries
Uncomment any of these examples to see the system in action:

In [ ]:
# Knowledge query examples
# process_user_query("What is a Health Savings Account?")
# process_user_query("Tell me about the benefits of FSA accounts")
# process_user_query("How do prepaid cards work?")

In [ ]:
# Card action query examples
# process_user_query("Please activate my card 1234-5678-9012-3456")
# process_user_query("I need to deactivate my card ending in 3456 because I lost it")

In [ ]:
# Ambiguous or unknown queries
# process_user_query("Can you help me with something?")
# process_user_query("What's the weather like today?") 